# Gold Layer Preparation for Power BI

`This notebook prepares aggregated and presentation-ready datasets from the Silver layer
for direct consumption in Power BI dashboards.`


In [0]:
silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/Silver"
loan_df = spark.read.format("delta").load(f"{silver_path}/loan_enriched")
display(loan_df)


`This section prepares a Gold-level loan dataset designed for Power BI consumption.`\
`It derives business-friendly segmentation buckets (vintage, income, credit score, exposure, term, pricing) and selects a curated set of fields for dashboarding.`\
`Gold tables are presentation-ready: they keep the Silver business logic and expose standardized categories to simplify BI slicing and filtering.`

In [0]:

df = loan_df
#derived features (segmentation buckets)

df_gold = (
    df
    # Vintage: origination year (common risk reporting segmentation)

    .withColumn("vintage_year", F.year("origination_date"))
    
    # Age bucket
    .withColumn(
        "age_bucket",
        F.when(F.col("age") < 26, "18-25")
         .when((F.col("age") >= 26) & (F.col("age") <= 40), "26-40")
         .when((F.col("age") > 40) & (F.col("age") <= 60), "41-60")
         .otherwise("60+")
    )
    
    # Income buckets: simplified segmentation for risk interpretation
    .withColumn(
        "income_bucket",
        F.when(F.col("annual_income") < 25000, "LOW")
         .when((F.col("annual_income") >= 25000) & (F.col("annual_income") < 60000), "MEDIUM")
         .otherwise("HIGH")
    )
    
    # Credit score buckets: coarse creditworthiness segmentation

    .withColumn(
        "credit_score_bucket",
        F.when(F.col("credit_score") < 600, "LOW")
         .when((F.col("credit_score") >= 600) & (F.col("credit_score") < 750), "MEDIUM")
         .otherwise("HIGH")
    )
    
    # Exposure buckets: loan size segmentation based on principal amount
    .withColumn(
        "exposure_bucket",
        F.when(F.col("principal_amount") < 5000, "<5K")
         .when((F.col("principal_amount") >= 5000) & (F.col("principal_amount") < 20000), "5K-20K")
         .when((F.col("principal_amount") >= 20000) & (F.col("principal_amount") < 100000), "20K-100K")
         .otherwise("100K+")
    )
    
    # Interest rate buckets (interest_rate_annual assumed as decimal, e.g. 0.05)
    .withColumn(
        "interest_rate_bucket",
        F.when(F.col("interest_rate_annual") < 0.03, "<3%")
         .when((F.col("interest_rate_annual") >= 0.03) & (F.col("interest_rate_annual") < 0.06), "3%-6%")
         .when((F.col("interest_rate_annual") >= 0.06) & (F.col("interest_rate_annual") < 0.10), "6%-10%")
         .otherwise("10%+")
    )
    
    # Term buckets: maturity segmentation
    .withColumn(
        "term_bucket",
        F.when(F.col("maturity_months") <= 12, "≤12m")
         .when((F.col("maturity_months") > 12) & (F.col("maturity_months") <= 36), "12-36m")
         .when((F.col("maturity_months") > 36) & (F.col("maturity_months") <= 84), "36-84m")
         .otherwise(">84m")
    )
    
    # Loan-to-income ratio: simple affordability indicator
    .withColumn(
        "loan_to_income",
        (F.col("principal_amount") / F.col("annual_income")).cast("double")
    )
    
    # Arrears buckets (based on avg_past_arrears)
    .withColumn(
        "arrears_bucket",
        F.when(F.col("avg_past_arrears") == 0, "NO_ARREARS")
         .when(F.col("avg_past_arrears") < 0.3, "LOW_ARREARS")
         .when(F.col("avg_past_arrears") < 0.7, "MEDIUM_ARREARS")
         .otherwise("HIGH_ARREARS")
    )
    
    # Simple high-risk flag: highlights an extreme-risk combination for BI focus
    .withColumn(
        "high_risk_flag",
        F.when(
            (F.col("income_bucket") == "LOW") &
            (F.col("credit_score_bucket") == "LOW") &
            (F.col("product_type") == "REVOLVING"),
            F.lit(1)
        ).otherwise(F.lit(0))
    )
)

# 2) we select curated columns for GOLD 
# we keep identifiers, core loan/customer/product attributes, risk fields and derived buckets.


cols_gold = [
    # IDs
    "loan_id", "product_id", "customer_id",
    
    # Loan information
    "origination_date", "principal_amount", "maturity_months", "interest_rate_annual",
    
    # Client
    "age", "age_bucket",
    "annual_income", "income_bucket",
    "employment_status",
    "nb_past_loans", "has_previous_defaults", "avg_past_arrears", "arrears_bucket",
    "credit_score", "credit_score_bucket", "risk_score_raw",
    
    # Product
    "product_name", "product_type", "regulatory_portfolio",
    "ifrs9_segment", "interest_rate_type", "base_margin_bp",
    "typical_maturity_min_months", "typical_maturity_max_months",
    "typical_amount_min", "typical_amount_max",
    "collateral_type", "secured_flag",
    "base_lgd_level", "lgd_sensitivity_to_macro",
    "ead_profile", "ccf_baseline",
    
    # Risk
    "default_flag", "default_date",
    "months_since_origination_at_default", "default_reason",
    
    # Gold derivatives
    "vintage_year",
    "exposure_bucket",
    "interest_rate_bucket",
    "term_bucket",
    "loan_to_income",
    "high_risk_flag"
]

loan_gold = df_gold.select(*[c for c in cols_gold if c in df_gold.columns])



In [0]:
# gold_path 


In [0]:
from pyspark.sql.functions import col, sum

null_counts = loan_gold.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in loan_gold.columns
])

display(null_counts)


loan_id,product_id,customer_id,origination_date,principal_amount,maturity_months,interest_rate_annual,age,age_bucket,annual_income,income_bucket,employment_status,nb_past_loans,has_previous_defaults,avg_past_arrears,arrears_bucket,credit_score,credit_score_bucket,risk_score_raw,product_name,product_type,regulatory_portfolio,ifrs9_segment,interest_rate_type,base_margin_bp,typical_maturity_min_months,typical_maturity_max_months,typical_amount_min,typical_amount_max,collateral_type,secured_flag,base_lgd_level,lgd_sensitivity_to_macro,ead_profile,ccf_baseline,default_flag,default_date,months_since_origination_at_default,default_reason,vintage_year,exposure_bucket,interest_rate_bucket,term_bucket,loan_to_income,high_risk_flag
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,127569,127569,127569,127569,0,0,0,0,0,0


In [0]:
df_gold = df_gold.withColumn(
    "default_flag",
    F.when(F.col("default_flag").isNull(), F.lit(0)).otherwise(F.col("default_flag"))
)


In [0]:
# verification

loan_gold.filter(F.col("loan_id").isNull()).count()
loan_gold.filter(F.col("default_flag").isNull()).count()


127569

In [0]:
#The loan_gold dataset is written in Delta format to the designated container

loan_gold.write.format("delta") \
    .mode("overwrite") \
    .save(f"{gold_path}/loan_gold")


In [0]:
gold_export_path = f"abfss://ifrsdatalake@{storage_account}.dfs.core.windows.net/Gold/loan_gold_parquet"

loan_gold.write \
    .mode("overwrite") \
    .parquet(gold_export_path)


In [0]:


(
    loan_gold
    .coalesce(1)  
    .write
    .mode("overwrite")
    .parquet(gold_export_path)
)
 